In [1]:
import pandas as pd

In [9]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [23]:
property_df = pd.read_csv("HomeHarvest_Chicago.csv")


In [25]:
# Assuming your DataFrame is named 'final_df'
columns_to_remove = ['primary_photo', 'alt_photos', 'mls', 'status', 'unit', 'city', 'state']

# Use the 'drop' method to remove the specified columns
property_df = property_df.drop(columns=columns_to_remove)


In [14]:
property_df.shape

(46014, 27)

In [26]:
property_df.head(5)

property_url      mls_id  \
0  https://www.realtor.com/realestateandhomes-detail/7797244224  11456282.0   
1  https://www.realtor.com/realestateandhomes-detail/7133125936  11429768.0   
2  https://www.realtor.com/realestateandhomes-detail/8298921491  11358564.0   
3  https://www.realtor.com/realestateandhomes-detail/7695426483  11468987.0   
4  https://www.realtor.com/realestateandhomes-detail/7002373377  11435377.0   

           style               street  zip_code  beds  full_baths  half_baths  \
0  SINGLE_FAMILY     6127 S Tripp Ave   60629.0   3.0         1.0         NaN   
1  SINGLE_FAMILY  6238 S Mayfield Ave   60638.0   4.0         2.0         1.0   
2         CONDOS    400 E Randolph St   60601.0   0.0         1.0         NaN   
3         CONDOS  6121 N Winthrop Ave   60660.0   3.0         2.0         1.0   
4         CONDOS    2838 W Addison St   60618.0   1.0         1.0         NaN   

     sqft  year_built  days_on_mls  list_price   list_date  sold_price  \
0   912.0      1945.0         53.0    239000.0  2022-07-07      214500   
1  1712.0      1926.0         81.0    329000.0  2022-06-09      335000   
2   650.0      1964.0        154.0    220000.0  2022-03-28      210000   
3     NaN      2008.0         41.0    635000.0  2022-07-19      625000   
4   750.0      1930.0         75.0    119900.0  2022-06-15      102750   

  last_sold_date  lot_sqft  price_per_sqft   latitude  longitude  stories  \
0     2022-08-29       NaN           235.0  41.781364 -87.728592      1.0   
1     2022-08-29    3781.0           196.0  41.778587 -87.769427      2.0   
2     2022-08-29       NaN           323.0  41.884987 -87.616671     40.0   
3     2022-08-29       NaN             NaN  41.993031 -87.657910      4.0   
4     2022-08-29       NaN           137.0  41.946948 -87.699666      3.0   

   hoa_fee  parking_garage  
0      NaN             2.0  
1      NaN             2.0  
2    430.0             1.0  
3    343.0             1.0  
4    185.0             NaN

In [24]:
property_df.status.value_counts()

status
SOLD    46014
Name: count, dtype: int64

In [15]:
missing_mls_count = property_df['mls'].isna().sum()
print("Number of missing 'mls' values:", missing_mls_count)


Number of missing 'mls' values: 9131


In [7]:
property_df.columns

Index(['property_url', 'mls', 'mls_id', 'status', 'style', 'street', 'unit',
       'city', 'state', 'zip_code', 'beds', 'full_baths', 'half_baths', 'sqft',
       'year_built', 'days_on_mls', 'list_price', 'list_date', 'sold_price',
       'last_sold_date', 'lot_sqft', 'price_per_sqft', 'latitude', 'longitude',
       'stories', 'hoa_fee', 'parking_garage', 'primary_photo', 'alt_photos'],
      dtype='object')

In [8]:
property_df.dtypes

property_url       object
mls                object
mls_id            float64
status             object
style              object
street             object
unit               object
city               object
state              object
zip_code          float64
beds              float64
full_baths        float64
half_baths        float64
sqft              float64
year_built        float64
days_on_mls       float64
list_price        float64
list_date          object
sold_price          int64
last_sold_date     object
lot_sqft          float64
price_per_sqft    float64
latitude          float64
longitude         float64
stories           float64
hoa_fee           float64
parking_garage    float64
primary_photo      object
alt_photos         object
dtype: object

In [28]:
property_df['last_sold_date'] = pd.to_datetime(property_df['last_sold_date'])
property_df['list_date'] = pd.to_datetime(property_df['list_date'])


In [29]:
min_date = property_df['last_sold_date'].min()
max_date = property_df['last_sold_date'].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)


Minimum Date: 2022-08-24 00:00:00
Maximum Date: 2023-11-17 00:00:00


In [30]:
min_date = property_df['list_date'].min()
max_date = property_df['list_date'].max()

print("Minimum Date:", min_date)
print("Maximum Date:", max_date)


Minimum Date: 2017-12-17 00:00:00
Maximum Date: 2024-01-06 00:00:00


In [31]:
import geopandas as gpd

# Load the neighborhood boundaries data (assuming it's in a GeoJSON file)
neighborhoods = gpd.read_file('boundries-chicago.geojson')

# Load your property data (assuming it's in a DataFrame named 'property_data')
property_df['geometry'] = gpd.points_from_xy(property_df['longitude'], property_df['latitude'])
property_df = gpd.GeoDataFrame(property_df, geometry='geometry', crs='EPSG:4326')

# Perform a spatial join
properties_with_neighborhood = gpd.sjoin(property_df, neighborhoods, op='within')

# Now, properties_with_neighborhood DataFrame will have a new column indicating the neighborhood


/Users/rashidbaset/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3466: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [33]:
properties_with_neighborhood.head(5)

property_url      mls_id  \
0    https://www.realtor.com/realestateandhomes-detail/7797244224  11456282.0   
226  https://www.realtor.com/realestateandhomes-detail/7258775322  11453400.0   
271  https://www.realtor.com/realestateandhomes-detail/7269790632  11379220.0   
352  https://www.realtor.com/realestateandhomes-detail/9555077296         NaN   
725  https://www.realtor.com/realestateandhomes-detail/8722343426  11457989.0   

             style               street  zip_code  beds  full_baths  \
0    SINGLE_FAMILY     6127 S Tripp Ave   60629.0   3.0         1.0   
226  SINGLE_FAMILY  7232 S Lawndale Ave   60629.0   3.0         2.0   
271   MULTI_FAMILY  4405 W Marquette Rd   60629.0   4.0         2.0   
352  SINGLE_FAMILY       3936 W 60th St   60629.0   NaN         NaN   
725  SINGLE_FAMILY  6229 S Komensky Ave   60629.0   4.0         3.0   

     half_baths    sqft  year_built  days_on_mls  list_price  list_date  \
0           NaN   912.0      1945.0         53.0    239000.0 2022-07-07   
226         NaN  1212.0      1960.0         52.0    289900.0 2022-07-03   
271         NaN     NaN      1957.0        127.0    214900.0 2022-04-19   
352         NaN  1050.0      1894.0          NaN         NaN        NaT   
725         NaN  2200.0      1924.0         52.0    379000.0 2022-07-12   

     sold_price last_sold_date  lot_sqft  price_per_sqft   latitude  \
0        214500     2022-08-29       NaN           235.0  41.781364   
226      289000     2022-08-24    3750.0           238.0  41.761396   
271      217000     2022-08-24    3750.0             NaN  41.770996   
352      300000     2022-08-26    3125.0           286.0  41.784391   
725      375000     2022-09-02    3781.0           170.0  41.779588   

     longitude  stories  hoa_fee  parking_garage                    geometry  \
0   -87.728592      1.0      NaN             2.0  POINT (-87.72859 41.78136)   
226 -87.715314      NaN      NaN             2.0  POINT (-87.71531 41.76140)   
271 -87.732599      NaN      0.0             NaN  POINT (-87.73260 41.77100)   
352 -87.721927      1.0      NaN             NaN  POINT (-87.72193 41.78439)   
725 -87.723657      2.0      0.0             2.0  POINT (-87.72366 41.77959)   

     index_right  community area     shape_area perimeter area_num_1  \
0             62  WEST LAWN    0  82315301.6831         0         65   
226           62  WEST LAWN    0  82315301.6831         0         65   
271           62  WEST LAWN    0  82315301.6831         0         65   
352           62  WEST LAWN    0  82315301.6831         0         65   
725           62  WEST LAWN    0  82315301.6831         0         65   

    area_numbe comarea_id comarea      shape_len  
0           65          0       0  41934.4901309  
226         65          0       0  41934.4901309  
271         65          0       0  41934.4901309  
352         65          0       0  41934.4901309  
725         65          0       0  41934.4901309

In [34]:
properties_with_neighborhood.community.value_counts()

community
NEAR NORTH SIDE    4344
LAKE VIEW          3126
WEST TOWN          2340
LINCOLN PARK       2100
NEAR WEST SIDE     1761
                   ... 
PULLMAN             100
OAKLAND              65
FULLER PARK          43
BURNSIDE             39
RIVERDALE            17
Name: count, Length: 77, dtype: int64